In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pyprogressivex
from time import time

In [2]:
# Load the correspondences from file
correspondences = np.loadtxt('img/tless.txt')
n = len(correspondences)

imagePoints = np.float32([correspondences[i][0:2] for i in np.arange(n)]).reshape(-1,2)
worldPoints = np.float32([correspondences[i][2:5] for i in np.arange(n)]).reshape(-1,3)

groundTruthPoses = np.loadtxt('img/tless_poses.txt')
intrinsicParams = np.loadtxt('img/tless_intrinsics.txt')
groundTruthModelNumber = groundTruthPoses.shape[0];

print("Number of correspondences loaded = ", str(len(correspondences)))
print("Number of ground truth poses loaded = ", str(groundTruthModelNumber))

Number of correspondences loaded =  1886
Number of ground truth poses loaded =  2


In [3]:
def verify_pyprogressivex(imagePoints, worldPoints, intrinsicParams):    
    
    threshold = 4.0
    
    poses, labeling = pyprogressivex.find6DPoses(np.ascontiguousarray(imagePoints), np.ascontiguousarray(worldPoints), np.ascontiguousarray(intrinsicParams), threshold)    
    return poses, labeling

def calculate_error(gt_pose, est_pose):
    
    R2R1 = np.dot(gt_pose[:, 0:3].T, est_pose[:, 0:3])
    cos_angle = max(-1.0, min(1.0, 0.5 * (R2R1.trace() - 1.0)))
    
    err_R = np.arccos(cos_angle) * 180.0 / np.pi
    err_t = np.linalg.norm(gt_pose[:, 3] - est_pose[:, 3])
    
    return err_R, err_t


In [7]:
t = time()

poses, labeling = verify_pyprogressivex(imagePoints, worldPoints, intrinsicParams)
model_number = poses.size / 12

print ('Time = ', time()-t, ' sec')
print ('Models found = {}'.format(model_number))

Time =  57.572970151901245  sec
Models found = 9.0


In [9]:
for i in np.arange(groundTruthModelNumber):
    gtPose = groundTruthPoses[i].reshape(3, 4)
    minRotationError = 1e10
    minTranslationError = 1e10
    
    for j in np.arange(model_number):
        
        estPose = poses[int(j * 3) : int((j + 1) * 3)]
    
        errR, errt = calculate_error(gtPose, estPose)
        
        if errR + errt < minRotationError + minTranslationError:
            minRotationError = errR
            minTranslationError = errt

    print ('Evaluating the {}. ground truth pose'.format(i + 1))
    print ('Rotation error = ', minRotationError, '°')
    print ('Translation error = ', minTranslationError / 10.0, ' cm\n')


Evaluating the 1. ground truth pose
Rotation error =  8.249138186163485 °
Translation error =  2.4035577603771756  cm

Evaluating the 2. ground truth pose
Rotation error =  0.9491160622888057 °
Translation error =  1.2155621577125257  cm

